In [33]:
import numpy as np
import pandas as pd

In [34]:
# import dataset
supermarket_data = pd.read_csv('supermarket_sales - Sheet1.csv', index_col=0)

In [35]:
# view dataset
supermarket_data.head()

,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
Invoice ID,,,,,,,,,,,,,,,,
750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,13:08,Ewallet,522.83,4.761905,26.1415,9.1
226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,3/8/2019,10:29,Cash,76.40,4.761905,3.8200,9.6
631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,13:23,Credit card,324.31,4.761905,16.2155,7.4
123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.2880,489.0480,1/27/2019,20:33,Ewallet,465.76,4.761905,23.2880,8.4
373-73-7910,A,Yangon,Normal,Male,Sports and travel,86.31,7,30.2085,634.3785,2/8/2019,10:37,Ewallet,604.17,4.761905,30.2085,5.3


In [36]:
# basic info
print('There are '
    f'{supermarket_data.shape[0]} rows and '
    f'{supermarket_data.shape[1]} columns '
    'in the supermarket_sales dataset.'
)
print('The datatypes in the supermarket_sales dataset are: ')
display(supermarket_data.dtypes)

There are 1000 rows and 16 columns in the supermarket_sales dataset.
The datatypes in the supermarket_sales dataset are: 


Branch                      object
City                        object
Customer type               object
Gender                      object
Product line                object
Unit price                 float64
Quantity                     int64
Tax 5%                     float64
Total                      float64
Date                        object
Time                        object
Payment                     object
cogs                       float64
gross margin percentage    float64
gross income               float64
Rating                     float64
dtype: object

Some features are inaccurate or unnecessary.
- Branch - Unnecessary (duplicate of City)
- Tax - Unnecessary (percentage of total)
- Total - Unnecessary (includes tax)
- cogs - inaccurate (total unit price, not COGS)
- gross margin percentage - inaccurate (same as tax column, as percentage of total)
- gross income - inaccurate (duplicate of tax)

Some features can be modified or engineered.
- Customer type - Boolean encoding to "member"
- Gender - Boolean encoding to "male"
- Date of purchase and Purchase time - Datetime encoding to "saledate"
- Unit price and Quantity - engineer "sale_gross"

In [57]:
# define features and datetime conversion
supermarket_data = pd.read_csv(
    'supermarket_sales - Sheet1.csv',
    usecols=[
        'Invoice ID',
        'City',
        'Customer type',
        'Gender',
        'Product line',
        'Unit price',
        'Quantity',
        'Payment',
        'Rating',
        'Date',
        'Time'
    ],
    parse_dates={'saledate': ['Date', 'Time']}
)

# modify features to boolean
supermarket_data['member'] = supermarket_data['Customer type'].map(
    {'Normal': 0, 'Member': 1}
)
supermarket_data['male'] = supermarket_data['Gender'].map(
    {'Female': 0, 'Male': 1}
)
supermarket_data = supermarket_data.drop(
    columns=[
        'Customer type',
        'Gender'
    ]
)

# engineer features
supermarket_data['sale_gross'] = (
    supermarket_data['Unit price'] * supermarket_data['Quantity']
)

# rename columns
supermarket_data = supermarket_data.rename(
    columns={
        'Invoice ID': 'invoice',
        'City': 'branch',
        'Product line': 'category',
        'Unit price': 'unit_price',
        'Quantity': 'quantity',
        'Payment': 'payment_type',
        'Rating': 'rating'
    }
)

In [58]:
supermarket_data.head()

,saledate,invoice,branch,category,unit_price,quantity,payment_type,rating,member,male,sale_gross
0,2019-01-05 13:08:00,750-67-8428,Yangon,Health and beauty,74.69,7,Ewallet,9.1,1,0,522.83
1,2019-03-08 10:29:00,226-31-3081,Naypyitaw,Electronic accessories,15.28,5,Cash,9.6,0,0,76.40
2,2019-03-03 13:23:00,631-41-3108,Yangon,Home and lifestyle,46.33,7,Credit card,7.4,0,1,324.31
3,2019-01-27 20:33:00,123-19-1176,Yangon,Health and beauty,58.22,8,Ewallet,8.4,1,1,465.76
4,2019-02-08 10:37:00,373-73-7910,Yangon,Sports and travel,86.31,7,Ewallet,5.3,0,1,604.17


KPIs:
- Characteristics of highest spenders
- Do members purchase more?
- What do members purchase most?
- What category has the highest sales?
- Which branch has the highest average rating?

Other questions:
- Is Payment related to Rating?
- Do electronic payments average higher sales than cash?